In [1]:
from package.sphincs import Sphincs

### Instantiate a $SPHINCS^+$ Object and Setting parameters ###

We will be using :
   - Security Parameter: $n=1$
   - Winternitz Parameter: $w=16$
   - Hypertree Height: $h=4$
   - Hypertree Layers: $d=2$
   - $FORS$ Trees Number: $k=4$
   - $FORS$ Trees Height: $a=2$

In [15]:
sphincs = Sphincs()

sphincs.set_n(2)
sphincs.set_h(4)
sphincs.set_d(2)
sphincs.set_k(4)
sphincs.set_a(2)

### Generating a Key Pair ###

In [16]:
sk, pk = sphincs.generate_key_pair()
print("Secret Key: ", sk)
print()
print("Public Key: ", pk)

Secret Key:  b'[\xa8%\x10\xb0\x9bM\xf0'

Public Key:  b'\xb0\x9bM\xf0'


### Signing M ###

In [17]:
m = b'Ripples of paradox spread out across the sea of causality.'

signature = sphincs.sign(m, sk)

print("Signature Size: ", len(signature))

Signature Size:  58


In [18]:
print("Is signature Correct ? ", sphincs.verify(signature, m, pk))

Is signature Correct ?  True


### Trying to find secret key with a Brute Force Attack on Secret Key###

In [21]:
sk_crack = bytes()

for i in range(0, 2 ** (sphincs._n * 8)):
    sk_crack = i.to_bytes(sphincs._n, 'big')  # Secret Key
    
    sk_crack += bytes(sphincs._n)   # Random Secret PRF, important to prevent forged messages from actual messages
                                    # But Because we are brute forcing Secret Key, messages are forged before
                                    # We don't need to create a really random one (0 is fine)
    sk_crack += pk  # Public Key
    
    sig_crack = sphincs.sign(m, sk_crack)  # Creating a signature
    
    if sphincs.verify(sig_crack, m, pk):  # Check if signature could be trust with the Public Key
        print("Secret Key Found: ", sk_crack, "\nWith main Private Seed: ", sk_crack[:sphincs._n])
        print("Cycles: ", i)
        break

print("\nDid we found the actual Private Seed? ", sk[:sphincs._n] == sk_crack[:sphincs._n])

if sk[:sphincs._n] != sk_crack[:sphincs._n]:
    print("We found a collision with the main seed!")

Secret Key Found:  b'\x12\xf1\x00\x00\xb0\x9bM\xf0' 
With main Private Seed:  b'\x12\xf1'
Cycles:  4849

Did we found the actual Private Seed?  False
We found a collision with the main seed!


### Trying to forge message with found Key ###

In [24]:
m2 = b'The pen is mightier than the sword ... if the sword is very short, and the pen is very sharp.'

signature2 = sphincs.sign(m2, sk_crack)

print(len(signature2))

58


In [25]:
print("Is signature Correct ? ", sphincs.verify(signature2, m, pk))

Is signature Correct ?  False


Our signature is wrong because we tried finding the secret key using only $m$ signature, with $m_2$, this kind of collision doesn't work !
We need to find the real secret key in order to forge our own message, and so test every possibilities!